In [16]:
!pip install ortools

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.7/27.7 MB 82.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.8/135.8 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 321.1/321.1 kB 19.9 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 5.29.5
    Uninstalling protobuf-5.29.5:
      Successfully uninstalled protobuf-5.29.5
  Attempting uninstall: absl-py
    Found existing installation: absl-py 1.4.0
    Uninstalling absl-py-1.4.0:
      Successfully uninstalled absl-py-1.4.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-ai-generativelanguage 0.6.15 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<6.0.0dev,>=3.20.2, but you have protobuf 6.31.1 which is incompatible.
grpcio-status 1.71.2 requires protobuf<6.0dev,>=5.26.1, but you have protobuf 6.31.1 w

In [1]:
!pip install folium

In [2]:
!pip install osrm

In [ ]:

import osrm
import requests
import json
import numpy as np

print("✅ OSRM libraries installed!")

✅ OSRM libraries installed!


In [33]:
locations = [
    ("Warehouse", 31.5204, 74.3587),  # central Lahore (depot)
    ("Shop A",    31.5340, 74.3572),
    ("Shop B",    31.5157, 74.3673),
    ("Shop C",    31.5280, 74.3810),
    ("Shop D",    31.5060, 74.3430),
    ("Shop E",    31.5400, 74.3440),
    ("Shop F",    31.5220, 74.3320),
    ("Shop G",    31.5100, 74.3790),
    ("Shop H",    31.5380, 74.3720),
]
demands = [0, 40, 20, 30, 10, 12, 15, 18, 20]

In [38]:
# Our delivery locations in Lahore
locations = [
    #("Warehouse", 31.5204, 74.3587),
    # central Lahore (depot)
    #31°31'20.4"N 72°34'35.4"E
    ("Warehouse", 31.522334, 72.576500),
    ('Shop 1', 31.570639, 72.889028),
    ('Shop 2', 31.591778, 72.836139),
    ('Shop 3', 31.495, 72.85875),
    ('Shop 4', 31.653, 72.865278),
    ('Shop 5', 31.677139, 72.909444),
    ('Shop 6', 31.480528, 72.652944),
    ('Shop 7', 31.431111, 72.745694),
    ('Shop 8', 31.644861, 72.818),
    ('Shop 9', 31.643861, 72.636389),
    ('Shop 10', 31.613639, 72.455833),
    ('Shop 11', 31.613806, 72.750056),
    ('Shop 12', 31.431444, 72.496167),
    ('Shop 13', 31.392056, 72.77875),
    ('Shop 14', 31.454667, 72.798083),
    ('Shop 15', 31.636528, 72.503222),
    ('Shop 16', 31.597333, 72.611944),
    ('Shop 17', 31.536472, 72.535139),
    ('Shop 18', 31.482861, 72.705389),
    ('Shop 19', 31.57825, 72.667722),
    ('Shop 20', 31.659056, 72.539472),
]

# Demands (warehouse = 0, shops need fertilizer bags)
demands = [0, 40, 20, 30, 10, 12, 15, 18, 20,0,0,0,0,0,0,0,0,0,0,0,0]


print("📍 Setup complete:")
print(f"  Locations: {len(locations)}")
print(f"  Total demand: {sum(demands)} bags")


📍 Setup complete:
  Locations: 21
  Total demand: 165 bags


In [39]:


# Vehicle fleet: (count, capacity in bags, cost_per_km)
# Example heterogenous fleet: 2 trucks and 2 vans
vehicles = [
    {"count": 1, "capacity": 200, "cost_per_km": 100.0},  # big truck: 1.0 currency/km
    {"count": 2, "capacity": 80,  "cost_per_km": 70},  # medium van
    {"count": 1, "capacity": 50,  "cost_per_km": 50},  # small van
]


In [40]:
# Step 2: Demands - how much each location needs
#demands = [0, 40, 20, 30, 10, 12, 15, 18, 20]  # warehouse needs 0, shops need fertilizer bags

print("\n📦 Demands (fertilizer bags):")
for i, (name, _, _) in enumerate(locations):
    print(f"  {name}: {demands[i]} bags")

total_demand = sum(demands)
print(f"\n📊 Total demand: {total_demand} bags")


📦 Demands (fertilizer bags):
  Warehouse: 0 bags
  Shop 1: 40 bags
  Shop 2: 20 bags
  Shop 3: 30 bags
  Shop 4: 10 bags
  Shop 5: 12 bags
  Shop 6: 15 bags
  Shop 7: 18 bags
  Shop 8: 20 bags
  Shop 9: 0 bags
  Shop 10: 0 bags
  Shop 11: 0 bags
  Shop 12: 0 bags
  Shop 13: 0 bags
  Shop 14: 0 bags
  Shop 15: 0 bags
  Shop 16: 0 bags
  Shop 17: 0 bags
  Shop 18: 0 bags
  Shop 19: 0 bags
  Shop 20: 0 bags

📊 Total demand: 165 bags


In [41]:
vehicle_specs = []
for v in vehicles:
    for _ in range(v["count"]):
        vehicle_specs.append((v["capacity"], v["cost_per_km"]))
num_vehicles = len(vehicle_specs)

In [42]:
from math import radians, sin, cos, sqrt, atan2
def haversine_km(lat1, lon1, lat2, lon2):
    """Calculate real-world distance between two GPS points"""
    R = 6371.0  # Earth's radius in km
    dlat = radians(lat2 - lat1)
    dlon = radians(lon2 - lon1)
    a = sin(dlat/2)**2 + cos(radians(lat1)) * cos(radians(lat2)) * sin(dlon/2)**2
    c = 2 * atan2(sqrt(a), sqrt(1-a))
    return R * c

In [43]:
n = len(locations)
dist_km = [[0]*n for _ in range(n)]
road_factor = 1.25  # roads are ~25% longer than straight-line distance

print("\n📏 Distance Matrix (km):")
print("     ", end="")
for i in range(n):
    print(f"{i:5}", end="")
print()

for i in range(n):
    print(f"{i:2}: ", end="")
    for j in range(n):
        if i == j:
            dist_km[i][j] = 0
            print(f"{0:5.1f}", end="")
        else:
            dist_km[i][j] = haversine_km(locations[i][1], locations[i][2],
                                         locations[j][1], locations[j][2]) * road_factor


📏 Distance Matrix (km):
         0    1    2    3    4    5    6    7    8    9   10   11   12   13   14   15   16   17   18   19   20
 0:   0.0 1:   0.0 2:   0.0 3:   0.0 4:   0.0 5:   0.0 6:   0.0 7:   0.0 8:   0.0 9:   0.010:   0.011:   0.012:   0.013:   0.014:   0.015:   0.016:   0.017:   0.018:   0.019:   0.020:   0.0

In [44]:
import requests
import numpy as np

In [45]:
# Setup OSRM client - using public demo server
# For production, you'd run your own OSRM server
OSRM_URL = "http://router.project-osrm.org"

def get_osrm_matrix(locations):
    """Get distance and duration matrix using OSRM"""
    print(f"🌐 Getting OSRM matrix for {len(locations)} locations...")

    # Prepare coordinates for OSRM (lon,lat format)
    coordinates = []
    for name, lat, lon in locations:
        coordinates.append([lon, lat])  # OSRM uses lon,lat

    # Build OSRM table request URL
    coord_string = ";".join([f"{lon},{lat}" for lon, lat in coordinates])
    url = f"{OSRM_URL}/table/v1/driving/{coord_string}"

    params = {
        'sources': ';'.join([str(i) for i in range(len(locations))]),
        'destinations': ';'.join([str(i) for i in range(len(locations))]),
        'annotations': 'distance,duration'
    }

    try:
        response = requests.get(url, params=params, timeout=30)
        response.raise_for_status()
        data = response.json()

        if data['code'] == 'Ok':
            # Extract distance matrix (meters) and duration matrix (seconds)
            distances = np.array(data['distances']) / 1000  # Convert to km
            durations = np.array(data['durations']) / 60     # Convert to minutes

            print(f"✅ OSRM matrix retrieved successfully!")
            return distances, durations
        else:
            print(f"❌ OSRM error: {data['message']}")
            return None, None

    except Exception as e:
        print(f"❌ OSRM request failed: {e}")
        return None, None

def get_osrm_route(start_coords, end_coords):
    """Get detailed route between two points"""
    try:
        coord_string = f"{start_coords[1]},{start_coords[0]};{end_coords[1]},{end_coords[0]}"
        url = f"{OSRM_URL}/route/v1/driving/{coord_string}"

        params = {
            'overview': 'full',
            'geometries': 'geojson'
        }

        response = requests.get(url, params=params, timeout=15)
        data = response.json()

        if data['code'] == 'Ok':
            route = data['routes'][0]
            geometry = route['geometry']['coordinates']
            # Convert to lat,lon for folium
            path = [[coord[1], coord[0]] for coord in geometry]
            return path, route['distance']/1000, route['duration']/60
        else:
            return None, None, None

    except Exception as e:
        print(f"Route error: {e}")
        return None, None, None

# Test OSRM with our locations
print("🧪 Testing OSRM connectivity...")
osrm_distances, osrm_durations = get_osrm_matrix(locations)

🧪 Testing OSRM connectivity...
🌐 Getting OSRM matrix for 21 locations...
✅ OSRM matrix retrieved successfully!


In [46]:
import folium

In [47]:
if osrm_distances is not None:
    print("\n📊 OSRM Distance Matrix (km):")
    print("     ", end="")
    for i in range(len(locations)):
        print(f"{i:6}", end="")
    print()

    for i in range(len(locations)):
        print(f"{i:2}: ", end="")
        for j in range(len(locations)):
            print(f"{osrm_distances[i][j]:6.1f}", end="")
        print(f"  ({locations[i][0]})")

    # Convert to OR-Tools integer matrix
    osrm_dist_matrix = [[int(round(d * 100)) for d in row] for row in osrm_distances]
    print("✅ Using OSRM real driving distances!")

    # Compare with haversine
    print("\n📈 OSRM vs Haversine Comparison:")
    total_osrm = np.sum(osrm_distances)
    total_haversine = sum(sum(row) for row in dist_km)
    difference = ((total_osrm - total_haversine) / total_haversine) * 100
    print(f"   OSRM total: {total_osrm:.1f} km")
    print(f"   Haversine total: {total_haversine:.1f} km")
    print(f"   Difference: {difference:+.1f}% (OSRM more accurate)")

else:
    print("⚠️ OSRM failed, using haversine distances")
    osrm_dist_matrix = dist_matrix
    osrm_distances = dist_km


📊 OSRM Distance Matrix (km):
          0     1     2     3     4     5     6     7     8     9    10    11    12    13    14    15    16    17    18    19    20
 0:    0.0  37.4  30.8  32.1  32.8  36.4  15.0  24.1  26.9  20.4  33.7  19.6  17.6  29.3  31.1  28.4  15.6   5.3  17.2  10.8  26.0  (Warehouse)
 1:   37.4   0.0   6.6   9.2  11.5  17.3  37.5  24.7  14.8  32.2  50.8  17.8  55.0  26.9  17.4  45.5  32.7  42.7  30.5  26.6  43.1  (Shop 1)
 2:   30.8   6.6   0.0  11.6   8.7  14.5  37.0  27.3   8.1  25.6  44.2  11.2  48.4  29.6  20.1  38.9  26.0  36.1  22.7  20.0  36.5  (Shop 2)
 3:   32.1   9.2  11.6   0.0  20.3  26.1  28.5  15.7  19.8  33.9  47.3  21.1  49.6  18.0   8.4  41.9  29.1  37.4  21.5  24.3  39.5  (Shop 3)
 4:   32.8  11.5   8.7  20.3   0.0   5.8  39.0  43.1   5.9  27.6  46.2  13.2  50.4  48.4  28.7  40.8  28.0  38.1  36.2  22.0  38.4  (Shop 4)
 5:   36.4  17.3  14.5  26.1   5.8   0.0  42.6  46.7   9.5  31.2  49.8  16.8  54.0  52.0  34.5  44.4  31.6  41.7  39.8  25.6  42.0

In [48]:
# Setup VRP with OSRM distances
from ortools.constraint_solver import pywrapcp, routing_enums_pb2
manager = pywrapcp.RoutingIndexManager(len(osrm_dist_matrix), len(vehicle_specs), 0)
routing = pywrapcp.RoutingModel(manager)

# OSRM-based cost evaluators
for vid, (cap, cost_per_km) in enumerate(vehicle_specs):
    def make_osrm_cost_eval(multiplier):
        def cost_eval(from_index, to_index):
            f = manager.IndexToNode(from_index)
            t = manager.IndexToNode(to_index)
            return int(round(osrm_dist_matrix[f][t] / 100 * multiplier * 100))
        return cost_eval

    cost_evaluator = routing.RegisterTransitCallback(make_osrm_cost_eval(cost_per_km))
    routing.SetArcCostEvaluatorOfVehicle(cost_evaluator, vid)
def demand_callback(from_index):
    node = manager.IndexToNode(from_index)
    return demands[node]
# Add constraints
demand_cb_idx = routing.RegisterUnaryTransitCallback(demand_callback)
routing.AddDimensionWithVehicleCapacity(
    demand_cb_idx, 0, [spec[0] for spec in vehicle_specs], True, "Capacity")

for v in range(len(vehicle_specs)):
    routing.SetFixedCostOfVehicle(1000, v)

# Solve with OSRM data
search_parameters = pywrapcp.DefaultRoutingSearchParameters()
search_parameters.first_solution_strategy = routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC
search_parameters.time_limit.seconds = 15

print("\n🔍 Solving VRP with OSRM real driving distances...")
solution = routing.SolveWithParameters(search_parameters)


🔍 Solving VRP with OSRM real driving distances...


In [49]:
if solution:
    print("✅ OSRM-enhanced solution found!")

    # Extract optimized routes
    osrm_routes = []

    for v in range(len(vehicle_specs)):
        index = routing.Start(v)
        route = []
        load = 0
        route_cost = 0
        route_distance = 0

        while not routing.IsEnd(index):
            node = manager.IndexToNode(index)
            route.append(node)
            load += demands[node]
            next_index = solution.Value(routing.NextVar(index))
            route_cost += routing.GetArcCostForVehicle(index, next_index, v)
            if not routing.IsEnd(next_index):
                route_distance += osrm_distances[node][manager.IndexToNode(next_index)]
            index = next_index

        if len(route) > 1:  # Vehicle is used
            capacity, cost_per_km = vehicle_specs[v]

            # Get real driving paths for this route
            print(f"\n🛣️ Getting real paths for Vehicle {v+1}...")
            full_path = []

            for i in range(len(route)):
                current_loc = route[i]
                full_path.append([locations[current_loc][1], locations[current_loc][2]])

                if i < len(route) - 1:  # Not last location
                    next_loc = route[i + 1]
                    start_coords = (locations[current_loc][1], locations[current_loc][2])
                    end_coords = (locations[next_loc][1], locations[next_loc][2])

                    path, dist, duration = get_osrm_route(start_coords, end_coords)
                    if path:
                        # Add intermediate points (skip first point to avoid duplication)
                        full_path.extend(path[1:])
                    else:
                        # Fallback to direct line
                        full_path.append([locations[next_loc][1], locations[next_loc][2]])

            # Add return to warehouse
            if len(route) > 0:
                last_loc = route[-1]
                start_coords = (locations[last_loc][1], locations[last_loc][2])
                end_coords = (locations[0][1], locations[0][2])
                path, dist, duration = get_osrm_route(start_coords, end_coords)
                if path:
                    full_path.extend(path[1:])  # Skip first point
                else:
                    full_path.append([locations[0][1], locations[0][2]])

            osrm_routes.append({
                'vehicle_id': v + 1,
                'route': route,
                'full_path': full_path,
                'load': load,
                'capacity': capacity,
                'distance': route_distance,
                'cost': route_cost / 100,
                'cost_per_km': cost_per_km
            })

            # Display route
            route_names = [locations[idx][0] for idx in route]
            print(f"🚚 Vehicle {v+1} ({capacity} bags, ₨{cost_per_km}/km):")
            print(f"   Route: {' → '.join(route_names)} → Warehouse")
            print(f"   Load: {load}/{capacity} bags ({load/capacity*100:.1f}% full)")
            print(f"   Distance: {route_distance:.1f} km (OSRM real roads)")
            print(f"   Cost: ₨{route_cost/100:.2f}")

    print(f"\n📊 OSRM-ENHANCED SUMMARY:")
    total_cost = sum(r['cost'] for r in osrm_routes)
    total_distance = sum(r['distance'] for r in osrm_routes)
    print(f"   Vehicles used: {len(osrm_routes)}/{len(vehicle_specs)}")
    print(f"   Total distance: {total_distance:.1f} km (real driving)")
    print(f"   Total cost: ₨{total_cost:.2f}")
    print(f"   Cost per bag: ₨{total_cost/total_demand:.2f}")

else:
    print("❌ No OSRM solution found!")
    osrm_routes = []

✅ OSRM-enhanced solution found!

🛣️ Getting real paths for Vehicle 2...
🚚 Vehicle 2 (80 bags, ₨70/km):
   Route: Warehouse → Shop 2 → Shop 4 → Shop 5 → Shop 8 → Shop 11 → Shop 19 → Warehouse
   Load: 62/80 bags (77.5% full)
   Distance: 70.8 km (OSRM real roads)
   Cost: ₨5726.20

🛣️ Getting real paths for Vehicle 3...
🚚 Vehicle 3 (80 bags, ₨70/km):
   Route: Warehouse → Shop 1 → Shop 3 → Warehouse
   Load: 70/80 bags (87.5% full)
   Distance: 46.6 km (OSRM real roads)
   Cost: ₨5519.00

🛣️ Getting real paths for Vehicle 4...
🚚 Vehicle 4 (50 bags, ₨50/km):
   Route: Warehouse → Shop 17 → Shop 12 → Shop 6 → Shop 18 → Shop 7 → Shop 13 → Shop 14 → Shop 16 → Shop 20 → Shop 15 → Shop 10 → Shop 9 → Warehouse
   Load: 33/50 bags (66.0% full)
   Distance: 151.1 km (OSRM real roads)
   Cost: ₨8547.50

📊 OSRM-ENHANCED SUMMARY:
   Vehicles used: 3/4
   Total distance: 268.6 km (real driving)
   Total cost: ₨19792.70
   Cost per bag: ₨119.96


*Prompted all locations*
✅ OSRM-enhanced solution found!

🛣️ Getting real paths for Vehicle 2...
🚚 Vehicle 2 (80 bags, ₨70/km):
   Route: Warehouse → Shop 2 → Shop 4 → Shop 5 → Shop 8 → Shop 11 → Shop 19 → Warehouse
   Load: 62/80 bags (77.5% full)
   Distance: 70.8 km (OSRM real roads)
   Cost: ₨5726.20

🛣️ Getting real paths for Vehicle 3...
🚚 Vehicle 3 (80 bags, ₨70/km):
   Route: Warehouse → Shop 1 → Shop 3 → Warehouse
   Load: 70/80 bags (87.5% full)
   Distance: 46.6 km (OSRM real roads)
   Cost: ₨5519.00

🛣️ Getting real paths for Vehicle 4...
🚚 Vehicle 4 (50 bags, ₨50/km):
   Route: Warehouse → Shop 17 → Shop 12 → Shop 6 → Shop 18 → Shop 7 → Shop 13 → Shop 14 → Shop 16 → Shop 20 → Shop 15 → Shop 10 → Shop 9 → Warehouse
   Load: 33/50 bags (66.0% full)
   Distance: 151.1 km (OSRM real roads)
   Cost: ₨8547.50

📊 OSRM-ENHANCED SUMMARY:
   Vehicles used: 3/4
   Total distance: 268.6 km (real driving)
   Total cost: ₨19792.70
   Cost per bag: ₨119.96

In [ ]:
if solution:
    print("✅ OSRM-enhanced solution found!")

    # Extract optimized routes
    osrm_routes = []

    for v in range(len(vehicle_specs)):
        index = routing.Start(v)
        route = []
        load = 0
        route_cost = 0
        route_distance = 0

        while not routing.IsEnd(index):
            node = manager.IndexToNode(index)
            route.append(node)
            load += demands[node]
            next_index = solution.Value(routing.NextVar(index))
            route_cost += routing.GetArcCostForVehicle(index, next_index, v)
            if not routing.IsEnd(next_index):
                route_distance += osrm_distances[node][manager.IndexToNode(next_index)]
            index = next_index

        if len(route) > 1:  # Vehicle is used
            capacity, cost_per_km = vehicle_specs[v]

            # Get real driving paths for this route
            print(f"\n🛣️ Getting real paths for Vehicle {v+1}...")
            full_path = []

            for i in range(len(route)):
                current_loc = route[i]
                full_path.append([locations[current_loc][1], locations[current_loc][2]])

                if i < len(route) - 1:  # Not last location
                    next_loc = route[i + 1]
                    start_coords = (locations[current_loc][1], locations[current_loc][2])
                    end_coords = (locations[next_loc][1], locations[next_loc][2])

                    path, dist, duration = get_osrm_route(start_coords, end_coords)
                    if path:
                        # Add intermediate points (skip first point to avoid duplication)
                        full_path.extend(path[1:])
                    else:
                        # Fallback to direct line
                        full_path.append([locations[next_loc][1], locations[next_loc][2]])

            # Add return to warehouse
            if len(route) > 0:
                last_loc = route[-1]
                start_coords = (locations[last_loc][1], locations[last_loc][2])
                end_coords = (locations[0][1], locations[0][2])
                path, dist, duration = get_osrm_route(start_coords, end_coords)
                if path:
                    full_path.extend(path[1:])  # Skip first point
                else:
                    full_path.append([locations[0][1], locations[0][2]])

            osrm_routes.append({
                'vehicle_id': v + 1,
                'route': route,
                'full_path': full_path,
                'load': load,
                'capacity': capacity,
                'distance': route_distance,
                'cost': route_cost / 100,
                'cost_per_km': cost_per_km
            })

            # Display route
            route_names = [locations[idx][0] for idx in route]
            print(f"🚚 Vehicle {v+1} ({capacity} bags, ₨{cost_per_km}/km):")
            print(f"   Route: {' → '.join(route_names)} → Warehouse")
            print(f"   Load: {load}/{capacity} bags ({load/capacity*100:.1f}% full)")
            print(f"   Distance: {route_distance:.1f} km (OSRM real roads)")
            print(f"   Cost: ₨{route_cost/100:.2f}")

    print(f"\n📊 OSRM-ENHANCED SUMMARY:")
    total_cost = sum(r['cost'] for r in osrm_routes)
    total_distance = sum(r['distance'] for r in osrm_routes)
    print(f"   Vehicles used: {len(osrm_routes)}/{len(vehicle_specs)}")
    print(f"   Total distance: {total_distance:.1f} km (real driving)")
    print(f"   Total cost: ₨{total_cost:.2f}")
    print(f"   Cost per bag: ₨{total_cost/total_demand:.2f}")

else:
    print("❌ No OSRM solution found!")
    osrm_routes = []

✅ OSRM-enhanced solution found!

🛣️ Getting real paths for Vehicle 2...
🚚 Vehicle 2 (80 bags, ₨70/km):
   Route: Warehouse → Shop 2 → Shop 4 → Shop 5 → Shop 8 → Shop 11 → Shop 19 → Warehouse
   Load: 62/80 bags (77.5% full)
   Distance: 70.8 km (OSRM real roads)
   Cost: ₨5726.20

🛣️ Getting real paths for Vehicle 3...
🚚 Vehicle 3 (80 bags, ₨70/km):
   Route: Warehouse → Shop 1 → Shop 3 → Warehouse
   Load: 70/80 bags (87.5% full)
   Distance: 46.6 km (OSRM real roads)
   Cost: ₨5519.00

🛣️ Getting real paths for Vehicle 4...
🚚 Vehicle 4 (50 bags, ₨50/km):
   Route: Warehouse → Shop 17 → Shop 12 → Shop 6 → Shop 18 → Shop 7 → Shop 13 → Shop 14 → Shop 16 → Shop 20 → Shop 15 → Shop 10 → Shop 9 → Warehouse
   Load: 33/50 bags (66.0% full)
   Distance: 151.1 km (OSRM real roads)
   Cost: ₨8547.50

📊 OSRM-ENHANCED SUMMARY:
   Vehicles used: 3/4
   Total distance: 268.6 km (real driving)
   Total cost: ₨19792.70
   Cost per bag: ₨119.96


In [ ]:
if solution:
    print("✅ OSRM-enhanced solution found!")

    # Extract optimized routes
    osrm_routes = []

    for v in range(len(vehicle_specs)):
        index = routing.Start(v)
        route = []
        load = 0
        route_cost = 0
        route_distance = 0

        while not routing.IsEnd(index):
            node = manager.IndexToNode(index)
            route.append(node)
            load += demands[node]
            next_index = solution.Value(routing.NextVar(index))
            route_cost += routing.GetArcCostForVehicle(index, next_index, v)
            if not routing.IsEnd(next_index):
                route_distance += osrm_distances[node][manager.IndexToNode(next_index)]
            index = next_index

        if len(route) > 1:  # Vehicle is used
            capacity, cost_per_km = vehicle_specs[v]

            # Get real driving paths for this route
            print(f"\n🛣️ Getting real paths for Vehicle {v+1}...")
            full_path = []

            for i in range(len(route)):
                current_loc = route[i]
                full_path.append([locations[current_loc][1], locations[current_loc][2]])

                if i < len(route) - 1:  # Not last location
                    next_loc = route[i + 1]
                    start_coords = (locations[current_loc][1], locations[current_loc][2])
                    end_coords = (locations[next_loc][1], locations[next_loc][2])

                    path, dist, duration = get_osrm_route(start_coords, end_coords)
                    if path:
                        # Add intermediate points (skip first point to avoid duplication)
                        full_path.extend(path[1:])
                    else:
                        # Fallback to direct line
                        full_path.append([locations[next_loc][1], locations[next_loc][2]])

            # Add return to warehouse
            if len(route) > 0:
                last_loc = route[-1]
                start_coords = (locations[last_loc][1], locations[last_loc][2])
                end_coords = (locations[0][1], locations[0][2])
                path, dist, duration = get_osrm_route(start_coords, end_coords)
                if path:
                    full_path.extend(path[1:])  # Skip first point
                else:
                    full_path.append([locations[0][1], locations[0][2]])

            osrm_routes.append({
                'vehicle_id': v + 1,
                'route': route,
                'full_path': full_path,
                'load': load,
                'capacity': capacity,
                'distance': route_distance,
                'cost': route_cost / 100,
                'cost_per_km': cost_per_km
            })

            # Display route
            route_names = [locations[idx][0] for idx in route]
            print(f"🚚 Vehicle {v+1} ({capacity} bags, ₨{cost_per_km}/km):")
            print(f"   Route: {' → '.join(route_names)} → Warehouse")
            print(f"   Load: {load}/{capacity} bags ({load/capacity*100:.1f}% full)")
            print(f"   Distance: {route_distance:.1f} km (OSRM real roads)")
            print(f"   Cost: ₨{route_cost/100:.2f}")

    print(f"\n📊 OSRM-ENHANCED SUMMARY:")
    total_cost = sum(r['cost'] for r in osrm_routes)
    total_distance = sum(r['distance'] for r in osrm_routes)
    print(f"   Vehicles used: {len(osrm_routes)}/{len(vehicle_specs)}")
    print(f"   Total distance: {total_distance:.1f} km (real driving)")
    print(f"   Total cost: ₨{total_cost:.2f}")
    print(f"   Cost per bag: ₨{total_cost/total_demand:.2f}")

else:
    print("❌ No OSRM solution found!")
    osrm_routes = []

✅ OSRM-enhanced solution found!

🛣️ Getting real paths for Vehicle 2...
🚚 Vehicle 2 (80 bags, ₨70/km):
   Route: Warehouse → Shop 2 → Shop 4 → Shop 5 → Shop 8 → Shop 11 → Shop 19 → Warehouse
   Load: 62/80 bags (77.5% full)
   Distance: 70.8 km (OSRM real roads)
   Cost: ₨5726.20

🛣️ Getting real paths for Vehicle 3...
🚚 Vehicle 3 (80 bags, ₨70/km):
   Route: Warehouse → Shop 1 → Shop 3 → Warehouse
   Load: 70/80 bags (87.5% full)
   Distance: 46.6 km (OSRM real roads)
   Cost: ₨5519.00

🛣️ Getting real paths for Vehicle 4...
🚚 Vehicle 4 (50 bags, ₨50/km):
   Route: Warehouse → Shop 17 → Shop 12 → Shop 6 → Shop 18 → Shop 7 → Shop 13 → Shop 14 → Shop 16 → Shop 20 → Shop 15 → Shop 10 → Shop 9 → Warehouse
   Load: 33/50 bags (66.0% full)
   Distance: 151.1 km (OSRM real roads)
   Cost: ₨8547.50

📊 OSRM-ENHANCED SUMMARY:
   Vehicles used: 3/4
   Total distance: 268.6 km (real driving)
   Total cost: ₨19792.70
   Cost per bag: ₨119.96


In [50]:
def create_osrm_map(locations, routes, demands, filename="osrm_map.html"):
    """Create map with real OSRM driving routes and save as HTML"""
    center_lat = sum(loc[1] for loc in locations) / len(locations)
    center_lon = sum(loc[2] for loc in locations) / len(locations)

    m = folium.Map(location=[center_lat, center_lon], zoom_start=12)

    # Add warehouse
    folium.Marker(
        [locations[0][1], locations[0][2]],
        popup=f"🏭 {locations[0][0]}",
        icon=folium.Icon(color='red', icon='home', prefix='fa')
    ).add_to(m)

    # Add shops
    for i, (name, lat, lon) in enumerate(locations[1:], 1):
        folium.Marker(
            [lat, lon],
            popup=f"🏪 {name}<br>Demand: {demands[i]} bags",
            icon=folium.Icon(color='blue', icon='shopping-cart', prefix='fa')
        ).add_to(m)

    # Add real driving routes
    colors = ['red', 'blue', 'green', 'purple', 'orange', 'darkred']

    for i, route in enumerate(routes):
        color = colors[i % len(colors)]

        folium.PolyLine(
            route['full_path'],
            color=color,
            weight=5,
            opacity=0.8,
            popup=f"🚚 Vehicle {route['vehicle_id']}<br>"
                  f"Real driving route<br>"
                  f"Distance: {route['distance']:.1f} km<br>"
                  f"Cost: ₨{route['cost']:.2f}",
        ).add_to(m)

    # Enhanced legend
    legend_html = f'''
    <div style="position: fixed; bottom: 50px; left: 50px; width: 350px; height: 250px;
                background-color: white; border:2px solid grey; z-index:9999;
                font-size:14px; padding: 10px">
    <h4>🗺️ OSRM-Enhanced Route Map</h4>
    <p>🏭 <span style="color:red">Red</span>: Warehouse</p>
    <p>🏪 <span style="color:blue">Blue</span>: Customer Shops</p>
    <p>🛣️ <span style="color:purple">Colored Lines</span>: Real Driving Routes</p>
    <hr>
    <p><strong>Total Distance:</strong> {sum(r['distance'] for r in routes):.1f} km</p>
    <p><strong>Total Cost:</strong> ₨{sum(r['cost'] for r in routes):.2f}</p>
    <p><strong>Vehicles Used:</strong> {len(routes)}</p>
    <p><em>Powered by OSRM real-world routing</em></p>
    </div>
    '''

    m.get_root().html.add_child(folium.Element(legend_html))

    # Save and return file name

    return m


In [51]:

# Create and save enhanced map
if osrm_routes:
    print("🗺️ Creating OSRM-enhanced route map...")
    osrm_map = create_osrm_map(locations, osrm_routes, demands)
    osrm_map.save('osrm_optimized_routes.html')
    print("✅ OSRM map saved as 'osrm_optimized_routes.html'")

    # Display if possible
    try:
        osrm_map
    except:
        print("💡 Open 'osrm_optimized_routes.html' in your browser!")

🗺️ Creating OSRM-enhanced route map...
✅ OSRM map saved as 'osrm_optimized_routes.html'


# restructure